stuff to implement
* Get CCG token
* find metadata template
* /extract_structured endpoint

In [8]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_FOLDER_ID=os.getenv('EXERCISE5_FOLDER')
BOX_METADATA_TEMPLATE_KEY=os.getenv('METADATA_TEMPLATE_KEY')

In [2]:
from box_sdk_gen import BoxClient, CCGConfig, BoxCCGAuth

ccg_config = CCGConfig(
    client_id=BOX_CLIENT_ID,
    client_secret=BOX_CLIENT_SECRET,
    user_id=BOX_USER_ID,
)

ccg_auth = BoxCCGAuth(ccg_config)

client = BoxClient(ccg_auth)

print(f"{client.users.get_user_me()}")

<class 'box_sdk_gen.schemas.user_full.UserFull'> {'id': '19498290761', 'type': 'user', 'name': 'Scott Hurrey', 'login': 'shurrey+eplusadmin@boxdemo.com', 'created_at': '2022-05-26T10:57:52-07:00', 'modified_at': '2025-08-25T13:51:12-07:00', 'language': 'en', 'timezone': 'America/Los_Angeles', 'space_amount': 999999999999999, 'space_used': 3609201911, 'max_upload_size': 536870912000, 'status': 'active', 'job_title': '', 'phone': '', 'address': '', 'avatar_url': 'https://hurrey.app.box.com/api/avatar/large/19498290761'}


In [11]:
from box_sdk_gen import (
    AiItemBase,
    AiItemBaseTypeField,
    CreateAiExtractStructuredMetadataTemplate
)

files = client.folders.get_folder_items(BOX_FOLDER_ID).to_dict()

print("Processing files...")

for file in files["entries"]:
    box_ai_response = client.ai.create_ai_extract_structured(
        items=[
            AiItemBase(
                id=file["id"],
                type=AiItemBaseTypeField.FILE,
            )
        ],
        metadata_template=CreateAiExtractStructuredMetadataTemplate(
            scope="enterprise",
            template_key=BOX_METADATA_TEMPLATE_KEY
        )
    )

    print(f"File: {file['name']}")
    print(f"AI Extract Structured Metadata: {box_ai_response.answer}")
    print("--------------------------------------------------")

Processing files...
File: demo-invoice-20tax-1.pdf
AI Extract Structured Metadata: <class 'box_sdk_gen.schemas.ai_extract_response.AiExtractResponse'> {'companyName': 'Polychemtex Inc.', 'billedCompanyName': 'ACME Inc', 'invoiceNumber': '86864', 'invoiceDate': '2025-03-29T00:00:00Z', 'dueDate': '2025-04-28T00:00:00Z', 'orderNumber': '000037', 'total': 1287.4}
--------------------------------------------------
File: demo-invoice-20tax-10.pdf
AI Extract Structured Metadata: <class 'box_sdk_gen.schemas.ai_extract_response.AiExtractResponse'> {'companyName': 'Cloud VPS', 'billedCompanyName': 'ACME Inc', 'invoiceNumber': '405994', 'invoiceDate': '2025-04-08T00:00:00Z', 'dueDate': '2025-05-08T00:00:00Z', 'total': 50.82}
--------------------------------------------------
File: demo-invoice-20tax-2.pdf
AI Extract Structured Metadata: <class 'box_sdk_gen.schemas.ai_extract_response.AiExtractResponse'> {'companyName': 'Contoso Asia', 'billedCompanyName': 'ACME Inc', 'invoiceNumber': '347003', 'i

In [13]:
%%writefile box_ai_structured_extract.py
import os
import asyncio
from dotenv import load_dotenv

from box_sdk_gen import (
    BoxClient,
    CCGConfig,
    BoxCCGAuth,
    AiItemBase,
    AiItemBaseTypeField,
    CreateAiExtractStructuredMetadataTemplate
)

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_FOLDER_ID=os.getenv('EXERCISE5_FOLDER')
BOX_METADATA_TEMPLATE_KEY=os.getenv('METADATA_TEMPLATE_KEY')

def get_box_client():
    ccg_config = CCGConfig(
        client_id=BOX_CLIENT_ID,
        client_secret=BOX_CLIENT_SECRET,
        user_id=BOX_USER_ID,
    )

    ccg_auth = BoxCCGAuth(ccg_config)

    client = BoxClient(ccg_auth)

    return client

async def chat_with_ai(): 
    client = get_box_client()
    print(f"{client.users.get_user_me()}")
    
    files = client.folders.get_folder_items(BOX_FOLDER_ID).to_dict()

    print("Processing files...")

    for file in files["entries"]:
        box_ai_response = client.ai.create_ai_extract_structured(
            items=[
                AiItemBase(
                    id=file["id"],
                    type=AiItemBaseTypeField.FILE,
                )
            ],
            metadata_template=CreateAiExtractStructuredMetadataTemplate(
                scope="enterprise",
                template_key=BOX_METADATA_TEMPLATE_KEY
            )
        )

        print(f"File: {file['name']}")
        print(f"AI Extract Structured Metadata: {box_ai_response.answer}")
        print("--------------------------------------------------")

if __name__ == "__main__":
    asyncio.run(chat_with_ai())

Overwriting box_ai_structured_extract.py
